In [ ]:
!pip install easyocr
!pip install imutils
!pip install paho-mqtt

## Importação de Bibliotecas

In [98]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr
from paho.mqtt import client as mqtt_client
from random import randint
import time
import json


## Criação da Base de Dados para Tratamento

In [44]:
dataBase = ['image1.jpg', 'image2.jpg', 'image3.jpg', 'image4.jpg']

In [91]:
def leituraPlaca(a):
  img = cv2.imread(a)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  #plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))

  bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction
  edged = cv2.Canny(bfilter, 30, 200) #Edge detection
  #plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

  keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  contours = imutils.grab_contours(keypoints)
  contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

  location = None
  for contour in contours:
      approx = cv2.approxPolyDP(contour, 10, True)
      if len(approx) == 4:
          location = approx
          break


  location

  mask = np.zeros(gray.shape, np.uint8)
  new_image = cv2.drawContours(mask, [location], 0,255, -1)
  new_image = cv2.bitwise_and(img, img, mask=mask)

  #plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))

  (x,y) = np.where(mask==255)
  (x1, y1) = (np.min(x), np.min(y))
  (x2, y2) = (np.max(x), np.max(y))
  cropped_image = gray[x1:x2+1, y1:y2+1]

  #plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))

  reader = easyocr.Reader(['en'])
  result = reader.readtext(cropped_image)
  result

  text = result[0][-2]
  font = cv2.FONT_HERSHEY_SIMPLEX
  res = cv2.putText(img, text=text, org=(approx[0][0][0], approx[1][0][1]+60), fontFace=font, fontScale=1, color=(0,255,0), thickness=2, lineType=cv2.LINE_AA)
  res = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0,255,0),3)
  #plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))

  return text

In [88]:
conjuntoPlacas = []

In [ ]:
for i in range(len(dataBase)):
  placa = leituraPlaca(dataBase[i])
  conjuntoPlacas.append(placa)

In [ ]:
conjuntoPlacas

In [103]:
broker = 'broker.hivemq.com'
port = 1883
topic = "api/request"
# generate client ID with pub prefix randomly
client_id = 'ID'+str(randint(0,1000))
#username = 'your username'
#password = 'your password'
deviceId = 'D1'+str(randint(0,1000))

def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc==0:
            print("Conectado ao MQTT broker")
        else:
            print("Falha ao conectar, codigo de erro: %d", rc)
 
 
    client = mqtt_client.Client(client_id)
    #client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client
 
def publish(client):
    for k in range(len(dataBase)):
      placa = conjuntoPlacas[k]
      vaga = str(k)

      msg = "{\"action\":\"notification/insert\",\"deviceId\":\""+deviceId+"\",\"notification\":{\"notification\": \"placas\",\"parameters\":{\"placa\":" + placa + ",\"vaga\":" + vaga + "}}}"
      result = client.publish(topic,msg)
      msg_status = result[0]
      if msg_status ==0:
          print(f"Mensagem : {msg} enviada para o topico {topic}")
      else:
          print(f"Falha ao enviar a mensagem para o topico {topic}")
 

def main():
    client = connect_mqtt()
    client.loop_start()
    publish(client)
    #contagem = 0
    #while contagem < len(dataBase):
        #publish(client)
        #time.sleep(2)
        #contagem += 1
    

if __name__ == '__main__':
    main()

Mensagem : {"action":"notification/insert","deviceId":"D1625","notification":{"notification": "placas","parameters":{"placa":HR.26 BR.9044,"vaga":0}}} enviada para o topico api/request
Mensagem : {"action":"notification/insert","deviceId":"D1625","notification":{"notification": "placas","parameters":{"placa":ICOVIDT9,"vaga":1}}} enviada para o topico api/request
Mensagem : {"action":"notification/insert","deviceId":"D1625","notification":{"notification": "placas","parameters":{"placa":EBJY. 982],"vaga":2}}} enviada para o topico api/request
Mensagem : {"action":"notification/insert","deviceId":"D1625","notification":{"notification": "placas","parameters":{"placa":H982 FKL,"vaga":3}}} enviada para o topico api/request
